# Reading in Data

In [45]:
import random
import copy
from typing import Callable, List
import pandas as pd
import numpy  as np
from graph import Graph
from segment import Segment

In [46]:
# Reading Data
#prep_carry_matches = pd.read_csv('data/phase-two-matches.csv')
prep_carry_matches = pd.read_csv('data/current-matches.csv')
delivery_times     = pd.read_csv('data/delivery-times.csv')
timematrix         = pd.read_csv('data/time_matrix2.csv')
prep_carry_matches.replace({np.nan: None}, inplace=True)

# Getting Ready for Savings Algorithm
We need to create some functions and prepare the data before we can get started.

## Creating Segments
Here, we're building the segments based on the given matching guidelines. That is, if some prep site delivers to some carry in sites, that's one segment.

In [25]:
# Defining all the segments. Each segment has one prep site, one/two carry-in sites, and one timewindow
#
# Ex: Segment 0 -> Prep Site = 450 | Carry-Ins = [604,] | Time-Window = (6:15, 7:15)
#     Segment 1 -> Prep Site = 450 | Carry-Ins = [604,] | Time-Windwo = (9:30, 10:30)
segments = []
for i in prep_carry_matches.values.tolist():
    obj = Segment(int(i[0]), int(i[1]), int(i[2]) if i[2] else None)
    segments.append(obj)

## Creating Distance Function
This is to help us later. We have a csv file with the times from each school to another. So, to simplify accesing these times, we're creating a function, `distance(src, dst)` which takes in two school's three digit codes and returns the time it takes to go from one to the next.

In [26]:
# Creating a distance function from the distance matrix
# Given two school's 3 digit codes, it returns the distance in time between them
# Ex: distance(981, 604) = 35.736
origin      = list(timematrix['Origin'])
destination = list(timematrix['Destination'])
times       = list(timematrix['Travel Time'])

pairs = zip(origin, destination, times)
distance_dictionary = {}
for src, dst, time in pairs:
    distance_dictionary[(src, dst)] = time

def distance(src_, dst_):
    return distance_dictionary[(src_, dst_)]

## Creating Time Windows


This part is a bit trickier. We need to create time windows for each of the segments. These are going to be used when we execute the savings algorithm to make sure we compute feasible routes. The time window for each segment is defined to be 

~~~
(start = earliest time any carryin is visted, end = earliest time any carryin is left)
~~~

In [27]:
# Given a carryin site, it returns the schedule for it's breakfast and lunch
def time_lookup(carryin_: int):
    for i in delivery_times.values:
        if i[0] == carryin_:
            return list(i)
    raise Exception(f'carryin site {carryin_} not found')

In [28]:
# Given all segments (assuming they have one or two carry-in sites), it builds time windows for those segments 
# It returns a dictionary which maps segments to time windows
def window_builder(segments_: List[Segment]):
    windows = {}
    visited = {}
    for seg in segments_:
        for carry in [seg.carry1, seg.carry2]:
            if carry not in visited:
                visited[carry] = 1
            else:
                visited[carry] += 1
    for indx, seg in enumerate(segments_):
        starts = []
        ends = []
        for carry in [seg.carry1, seg.carry2]:
            if carry is None:
                continue
            time = time_lookup(carry)
            if visited[carry] == 2:
                starts.append(time[1])
                ends.append(time[2])
                visited[carry] -= 1
            else:
                starts.append(time[3])
                ends.append(time[4])
        windows[indx] = strict_window(starts, ends, seg)
    return windows 

In [29]:
# Given two start and end times as strings, it converts the earliest of each to datetime format and returns it.
def strict_window(starts_: List[str], ends_: List[str], seg_: Segment):
    for i in range(len(starts_)):
        if starts_[i] == 'DIA':
            starts_[i] =  '5:30 AM'
    if len(starts_) == 1 and len(ends_) == 1:
        early = pd.to_datetime(starts_[0]) - pd.Timedelta(distance(seg_.prep, seg_.carry1) + LOAD, unit='min')
        late  = pd.to_datetime(ends_[0]) - pd.Timedelta(distance(seg_.prep, seg_.carry1) + LOAD, unit='min')
        return (early, late)
    
    starts = [pd.to_datetime(i) for i in starts_]
    ends = [pd.to_datetime(i) for i in ends_]
    earlyindx = min(range(len(starts)), key=starts.__getitem__)
    lateindx = min(range(len(ends)), key=ends.__getitem__)
    dis_to_early = distance(seg_.carry2 if starts[earlyindx] else seg_.carry1, seg_.prep)
    dis_between = distance(seg_.carry1, seg_.carry2)
    early = starts[earlyindx] - pd.Timedelta(dis_to_early + LOAD, unit='min')
    late = ends[lateindx] - pd.Timedelta(dis_to_early + dis_between + LOAD + UNLOAD, unit='min')

    return(early, late)

## Creating Service Times
Here, we need to determine how long it takes to service every segment. 

- If we're visiting just one carryin site it's going to be 

        distance(prep, carry) + load + unload

 - If we're visiting two carryin sites, then it'll be 

        min(distance(prep,carry1), distance(prep,carry2)) + distance(carry1, carry2) + load + 2*unload

What does that mean? It's saying the service time is the time it takes to go from the prep site to the closest carryin site and then going to the next carry in. We also have to account for the time it takes to load and unload food. We have to load once for every segment and unload 

In [30]:
# Calculates the service time for a segment given loading and unloading times
def service_time_calc(segment_: Segment):
    to_car_1 = distance(segment_.prep, segment_.carry1)
    if segment_.carry2 is None:
        return to_car_1 + LOAD + UNLOAD
    to_car_2 = distance(segment_.prep, segment_.carry2)
    return min(to_car_1, to_car_2) + distance(segment_.carry1, segment_.carry2) + LOAD + 2*UNLOAD

## Updating Segments 
Up until now, we did a lot of prep work to make functions that will fill in the fields for our segments. Here, we're using those functions so each `Segment()` object has its corresponding time window and service times. 

In [31]:
# Adding time_window and service_time fields to every segment
def seg_builder(segments_: List[Segment]):
    seg_windows = window_builder(segments_)
    for indx,seg in enumerate(segments_):
        seg.time_window = seg_windows[indx]
        seg.service_time = service_time_calc(seg)

# Modified Savings Algorithm 
All the prep work is in place. We're ready to code the modified savings algorithm.

## Savings List
First up, we need to create a list of savings. That is, we need to figure out how much time we can save by merging two segments and putting them on the same route

In [32]:
# Creates a savings list for all segment pairs 
def savings_generator(segments_: List[Segment], depot_: int):
    savings = []
    for indx1, seg1 in enumerate(segments_):
        for indx2,seg2 in enumerate(segments_):
            if seg1 is seg2:
                continue
            save = savings_helper(seg1, seg2, depot_)
            savings.append((indx1, indx2, save))
    savings.sort(key=lambda tup: tup[2])
    return savings

In [33]:
def savings_helper(seg1_: Segment, seg2_: Segment, depot_: int):
    if seg1_.carry2 is None:
        cross = distance(seg1_.carry1, seg2_.prep)
    else:
        cross = min(distance(seg1_.carry1, seg2_.prep), distance(seg1_.carry2, seg2_.prep))
    return -cross

## Savings algorithm
Here, we'll actually run the algorithm. The code is super short, but there's a lot going on under the hood within the `Graph()` class. You can see how the class is implemented in the `graph.py` file.

Notice that we have two versions of the savings algorithm. One that uses randomization with 100 iterations and gives you the best solution and one with no randomization and gives you an answer based on a pure greedy heuristic. 

In [47]:
# Greedy w/ Randomization
def savings_algorithm(segments_: List[Segment], distance_: Callable[[int, int], float]):
    random.seed(0)
    i = float('inf')
    j = None
    for x in range(100):
        routes = Graph(segments_, distance_)
        savings = savings_generator(segments, 0)
        while savings:
            topop = max(0, random.randint(len(savings)-10, len(savings)-1))
            highest_savings = savings.pop(topop)
            routes.merge(highest_savings[0], highest_savings[1])
        if len(routes.get_routes()) < i:
            i = len(routes.get_routes())
            j = copy.copy(routes)
    return j

In [35]:
# Pure Greedy
def savings_algorithm(segments_: List[Segment], distance_: Callable[[int, int], float]):
    routes = Graph(segments_, distance_)
    savings = savings_generator(segments, 0)
    while savings:
        highest_savings = savings.pop()
        routes.merge(highest_savings[0], highest_savings[1])
    return routes

In [48]:
LOAD = 9
UNLOAD = 9
seg_builder(segments)
segments[12].service_time = 180
y = savings_algorithm(segments, distance)
x = y.get_routes()

In [49]:
for route in x:
    route.sort(key=lambda seg: (segments[seg].time_window[0], segments[seg].time_window[1]))
for indx,route in enumerate(x):
    print(f"\nROUTE {indx}")
    for seg in route:
        ref = segments[seg]
        print(f"{ref.prep},{ref.carry1},{ref.carry2}")


ROUTE 0
982,516,None
408,316,158
450,604,None
408,158,316
982,516,None
450,604,None

ROUTE 1
971,117,None
292,181,None
971,117,None
150,161,None
292,181,None

ROUTE 2
423,477,801
423,490,None
464,499,None
301,438,None
423,477,801
464,436,None
275,533,None
423,490,None
464,499,None

ROUTE 3
451,212,None
457,328,None
214,192,None
451,213,None
214,192,None
451,212,213
248,168,None

ROUTE 4
278,178,None
405,264,None
412,223,203
437,252,None
457,328,None
190,750,None

ROUTE 5
455,605,383
455,488,None
455,383,None
455,488,605
218,999,None

ROUTE 6
984,394,None

ROUTE 7
415,426,None
682,532,602
682,515,None
461,479,478
682,891,None
415,426,473
461,479,478
682,497,515
682,532,None
682,602,None

ROUTE 8
981,110,None
419,522,None
981,110,None
419,522,None
424,509,None

ROUTE 9
150,161,None
258,179,None
258,179,None
